## 1. |mp0rt

In [1]:
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
import datetime
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
df_list = pd.read_csv('listings.csv').set_index('id')
pd.set_option('display.max_columns', 500)

In [ ]:
list(df_list.columns)

## 2. Wr4ng|

    ## Questionable columns: 'host_since'; max_min, min_max repeated(?), 
        #listings_count repeats(?)
    ## NLP columns: 'name', 'description', 'neighborhood_overview', 
    # 'host_about'(?),         
    

In [6]:
def wrangle(X):
    """A function to clean our data"""
    
    #Make a copy of the df
    X = X.copy()

    ## DROP unwanted columns; HiCard, urls, empty, redundant, etc.           80>|
    drop_cols=['listing_url', 'scrape_id', 'last_scraped', 'name', 'description', 
               'neighborhood_overview', 'picture_url', 'host_url',
               'host_name', 'host_since', 'host_location', 'host_about', 
               'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 
               'neighbourhood', 'neighbourhood_group_cleansed', 'bathrooms', 
               'amenities', 'calendar_updated', 'has_availability',
               'calendar_last_scraped', 'minimum_minimum_nights', 
               'maximum_minimum_nights', 'minimum_maximum_nights', 
               'maximum_maximum_nights']
    X.drop(columns=drop_cols, inplace=True)
    
    ## A FUNC and execution thereof tallying number of contacts
    def num_contact(items):
        items = items.replace('[','').replace(']','').replace("'",'').split(',')
        return len(items)
    # 
    X['host_verifications'] = X['host_verifications'].apply(num_contact)
    
    ## Changing columns to BOOL vals     
    # Convert "t/f" columns to BOOl columns.
    def strToBool(text):
      if text == 't':
        return True
      else:
        return False
    X['host_has_profile_pic'] = X['host_has_profile_pic'].apply(strToBool)
    X['host_identity_verified'] = X['host_identity_verified'].apply(strToBool)
    X['host_is_superhost'] = X['host_is_superhost'].apply(strToBool)
    X['instant_bookable'] = X['instant_bookable'].apply(strToBool)
    
    # Convert license column to BOOL column.
    def licenseToBool(text):      
        if type(text) != float: 
            return True
        else:
            return False
    X['license'] = X['license'].apply(lambda x: licenseToBool(x))  
    
    
    ## Shorten/rename as neighbourhood_cleansed' column
    X = X.rename({'neighbourhood_cleansed' : 'neighborhood'}, axis=1)
    
    ### BATH
    ## Change Bathroom column name
    X = X.rename({'bathrooms_text' : 'num_bathroom'}, axis=1)
    # Get rid of the 2 null values in the bathrooms_text column
    X = X[X['num_bathroom'].isnull() == False]
    # Create a column to specify if baths are shared or not
    def parse_baths_shared(string):
        if 'shared' in string.lower(): # .lower because there is one value to remove
            return True                # "Shared Half Bath", we do
        else:
            return False
        
    X['shared_bath'] = X['num_bathroom'].apply(parse_baths_shared)
    # Parse floats from the bathrooms_text column
    def parse_baths(text):
        if 'half-bath' in text.lower():
            return 0.5

        text = text.split(' ')
        return float(text[0])

    X['num_baths'] = X['num_bathroom'].apply(parse_baths)
    X = X.drop(columns='num_bathroom')
    ### END BATH
    
    # TODO: Price to INT
    X['price'] = X['price'].apply(lambda x: float(x.replace('$',"").replace(',','')))
    
    def datestojd(dt_str):
        if type(dt_str) == float:
            return dt_str
        else:
            jul_date = pd.to_datetime(dt_str)
        
        return jul_date.to_julian_date()

    X['first_review'] = X['first_review'].apply(lambda x: datestojd(x))
    X['last_review'] = X['last_review'].apply(lambda x: datestojd(x))
        
    return X

In [8]:
df_profile = wrangle(df_list)

In [9]:
df_profile

,host_id,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighborhood,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,shared_bath,num_baths
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
177,615,within an hour,100%,100%,True,2,2,5,True,False,Washington Virginia Vale,39.697530,-104.925880,Entire guesthouse,Entire home/apt,2,NaN,0.0,68.0,1,400,1.0,400.0,0,0,0,0,79,32,4,2457635.5,2459141.5,96.0,10.0,10.0,10.0,10.0,10.0,9.0,True,True,1,1,0,0,1.56,False,1.0
360,666,within an hour,100%,97%,True,3,3,7,True,True,Highland,39.767030,-105.002560,Entire guesthouse,Entire home/apt,3,2.0,2.0,135.0,3,29,3.0,29.0,4,25,54,54,127,35,1,2458343.5,2459136.5,100.0,10.0,10.0,10.0,10.0,10.0,10.0,True,False,2,2,0,0,4.71,False,1.0
364,783,NaN,NaN,NaN,False,1,1,5,True,True,Five Points,39.765510,-104.979020,Entire loft,Entire home/apt,3,1.0,1.0,179.0,185,190,185.0,190.0,29,59,89,363,87,0,0,2454969.5,2457504.5,97.0,10.0,10.0,10.0,10.0,9.0,9.0,False,False,1,1,0,0,0.62,False,1.5
590,933,within an hour,100%,86%,True,2,2,7,True,True,North Park Hill,39.753610,-104.912370,Private room in house,Private room,3,1.0,1.0,54.0,1,300,1.0,300.0,0,0,0,0,608,22,0,2454919.5,2459093.5,97.0,10.0,10.0,10.0,10.0,10.0,10.0,True,False,2,0,2,0,4.31,True,1.0
592,933,within an hour,100%,86%,True,2,2,7,True,True,North Park Hill,39.754380,-104.911560,Private room in house,Private room,2,1.0,1.0,45.0,4,365,4.0,365.0,0,0,0,0,160,4,0,2455488.5,2459106.5,97.0,10.0,9.0,10.0,10.0,10.0,10.0,True,False,2,0,2,0,1.31,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46086022,372429032,within an hour,100%,100%,False,6,6,5,True,True,Five Points,39.762060,-104.988760,Entire apartment,Entire home/apt,4,2.0,2.0,113.0,1,1125,1.0,1125.0,10,10,10,10,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,3,3,0,0,NaN,False,2.0
46087313,359071917,within a day,99%,85%,True,1124,1124,2,True,True,Highland,39.762160,-105.011350,Entire serviced apartment,Entire home/apt,5,2.0,2.0,205.0,90,365,90.0,365.0,0,0,2,277,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,191,191,0,0,NaN,False,2.0
46092129,224536130,within an hour,100%,97%,False,1,1,5,True,True,North Capitol Hill,39.744111,-104.987352,Entire apartment,Entire home/apt,2,1.0,0.0,200.0,1,28,1.0,1125.0,3,3,3,123,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1,1,0,0,NaN,False,1.0


## €dα


In [ ]:
profile = ProfileReport(df_profile)
profile.to_notebook_iframe()

# m0d3|
    - Should we encode(onehot/ordinal) some columns: 'host_response_time', 'neighborhood'(prob no), 
      'property_type', 'room_type'
    - DONE: target/train/test
    - DONE: Get a baseline accuracy
    - DONE: Update pipfile with imports, commented out, above
    - TODO: build pipeline
    - TODO: run models

In [11]:
df_model = wrangle(df_list)

# Train//tesT :: :: :: SΦLi_|_

In [12]:
# setting the target for 2! train/test splits
target = 'price'

y_init = df_model[target]
X_init = df_model.drop(columns=target)

In [13]:
print(X_init.shape)
print(y_init.shape)

(3151, 47)
(3151,)


In [14]:
X_subseq, X_test, y_subseq, y_test = train_test_split(X_init, y_init, test_size=0.1, random_state=13)

In [15]:
print(X_test.shape)

(316, 47)


In [16]:
print(X_subseq.shape)
print(y_subseq.shape)

(2835, 47)
(2835,)


In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_subseq, y_subseq, test_size=0.1114, random_state=17)

In [18]:
X_val.shape

(316, 47)

In [19]:
print(X_train.shape)
print(X_val.shape)

(2519, 47)
(316, 47)


## B4SSLI3N

In [20]:
baseline_pred = [y_train.mean()] * len(y_train)

baseline_mae = mean_absolute_error(y_train,baseline_pred)

print(baseline_mae)

82.89001996891902


## PIGH:P:LINE

### GrdSRCH M0de|

In [ ]:
pipe_01 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(random_state = 13,
                           n_jobs = -2)
)

# params are a DICT
params_01 = {
    # 'key__hyperparam'(which step in pipeline you want looked at) : 'value'     
    'randomforestregressor__max_depth' : range(200, 1201, 100),
    'randomforestregressor__n_estimators' :  range(50, 351, 10),
    'randomforestregressor__max_samples' : np.arange(0.1, 0.9, 0.1)
    }

In [ ]:
model_01 = GridSearchCV(
    pipe_01,
    param_grid = params_01,
    n_jobs = -2,
    cv = 5,
    verbose = 1
)

## Stripped Model

Currently top 10 importance features

In [21]:
# secondary model with top 10 feats

X_train2 = X_train[['host_id', 'last_review', 'minimum_nights', 'latitude',
                   'minimum_nights_avg_ntm', 'first_review', 'bedrooms',
                   'number_of_reviews', 'neighborhood', 'num_baths']]

X_val2 = X_val[['host_id', 'last_review', 'minimum_nights', 'latitude',
                'minimum_nights_avg_ntm', 'first_review', 'bedrooms',
                'number_of_reviews', 'neighborhood', 'num_baths']]

In [22]:
pipe_02 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(
        n_estimators = 250,
        max_depth = 150,
        max_samples = 0.5,
        random_state = 13,
        n_jobs = -2)
)

In [25]:
pipe_02.fit(X_train2, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['neighborhood'],
                                mapping=[{'col': 'neighborhood',
                                          'data_type': dtype('O'),
                                          'mapping': City Park West        1
Speer                 2
Skyland               3
Union Station         4
Congress Park         5
                     ..
Marston              74
Southmoor Park       75
Harvey Park South    76
Harvey Park          77
NaN                  -2
Length: 78, dtype: int64}])),
                ('simpleimputer', SimpleImputer()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=150, max_samples=0.5,
                                       n_estimators=250, n_jobs=-2,
                                       random_state=13))])

In [26]:
import joblib

filename = 'working_model.sav'
joblib.dump(pipe_02, filename)

['working_model.sav']

In [24]:
type(X_train2['last_review'].iloc[0])

numpy.float64

In [ ]:
model_01.fit(X_train, y_train)

In [ ]:
dir(model_01)

In [ ]:
model_01.error_score

In [ ]:
model_01.best_params_

'predict_proba'


### TODO: check cardinality/duplicates in neighborhood, property type, room_type (?)